In [ ]:
from tqdm import tqdm
import seaborn as sns
import networkx as nx
import pandas as pd
import numpy as np
import gensim
import spacy
import os
import gc
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from spacy.matcher import Matcher 
from collections import  Counter
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow as tf

In [ ]:
path="../input/CORD-19-research-challenge/"
data=pd.read_csv(path+"metadata.csv")

Checking for missing data

In [ ]:
data.isna().sum()

In [ ]:
stop=set(stopwords.words('english'))

def build_corpus(df,col="title"):
    corpus=[]
    lem=WordNetLemmatizer()
    stop=set(stopwords.words('english'))
    new= df[col].dropna().str.split()
    new=new.values.tolist()
    corpus=[lem.lemmatize(word.lower()) for i in new for word in i if(word) not in stop]
    
    return corpus

Most common words in the abstracts

In [ ]:
# x1=[]
# x2=[]
# corpus= build_corpus(data, "abstract")
# counter= Counter(corpus)
# common= counter.most_common()
# for word, count in common[:10]:
#     if(word not in stop):
#         x1.append(word)
#         x2.append(count)

# sns.barplot(x=x1, y=x2)

In [ ]:
# def prepare_similarity(vectors):
#     similarity=cosine_similarity(vectors)
#     return similarity

# def get_top_similar(sentence, sentence_list, similarity_matrix, topN):
#     # find the index of sentence in list
#     index = sentence_list.index(sentence)
#     # get the corresponding row in similarity matrix
#     similarity_row = np.array(similarity_matrix[index, :])
#     # get the indices of top similar
#     indices = similarity_row.argsort()[-topN:][::-1]
#     return [(i,sentence_list[i]) for i in indices]

In [ ]:
path="../input/cord-19-eda-parse-json-and-generate-clean-csv/"

In [ ]:
clean_comm=pd.read_csv(path+"clean_comm_use.csv",nrows=1000)
clean_comm['source']='clean_comm'
biox = pd.read_csv(path+"biorxiv_clean.csv")
biox['source']='biorx'

articles=pd.concat([biox,clean_comm])

In [ ]:
del biox,clean_comm
gc.collect()

In [ ]:
articles.shape

In [ ]:
articles.head()

In [ ]:
articles.shape

In [ ]:
articles.fillna("Unknown",inplace=True)

In [ ]:
# titles=[]
texts=[]
for i in range(100):
#     titles.append(articles.iloc[i]['title'])
    texts.append(articles.iloc[i]['text'])

In [ ]:
import re
def clean(txt):
    txt=re.sub(r'\n',' ',txt)
    txt=re.sub(r'\([^()]*\)',' ',txt)
    txt=re.sub(r'https?:\S+\sdoi',' ',txt)
    return txt

In [ ]:
texts=list(map(clean,texts))
text_list=' '.join(texts)

In [ ]:
print(text_list)

**Constructing knowldege graphs**

In [ ]:
import spacy
nlp=spacy.load('en_core_web_sm')

In [ ]:
# def get_entities(sent):
#     ## chunk 1
#     ent1 = ""
#     ent2 = ""

#     prv_tok_dep = ""    # dependency tag of previous token in the sentence
#     prv_tok_text = ""   # previous token in the sentence

#     prefix = ""
#     modifier = ""

#   #############################################################
  
#     for tok in nlp(sent):
#         ## chunk 2
#         # if token is a punctuation mark then move on to the next token
#         if tok.dep_ != "punct":
#           # check: token is a compound word or not
#           if tok.dep_ == "compound":
#             prefix = tok.text
#             # if the previous word was also a 'compound' then add the current word to it
#             if prv_tok_dep == "compound":
#                    prefix = prv_tok_text + " "+ tok.text
      
#       # check: token is a modifier or not
#         if tok.dep_.endswith("mod") == True:
#             modifier = tok.text
#             # if the previous word was also a 'compound' then add the current word to it
#             if prv_tok_dep == "compound":
#               modifier = prv_tok_text + " "+ tok.text

#           ## chunk 3
#         if tok.dep_.find("subj") == True:
#             ent1 = modifier +" "+ prefix + " "+ tok.text
#             prefix = ""
#             modifier = ""
#             prv_tok_dep = ""
#             prv_tok_text = ""      

#           ## chunk 4
#         if tok.dep_.find("obj") == True:
#             ent2 = modifier +" "+ prefix +" "+ tok.text

#           ## chunk 5  
#           # update variables
#         prv_tok_dep = tok.dep_
#         prv_tok_text = tok.text
#   #############################################################

#     return [ent1.strip(), ent2.strip()]

In [ ]:
# def get_relation(sent):

#   doc = nlp(sent)

#   # Matcher class object 
#   matcher = Matcher(nlp.vocab)

#   #define the pattern 
#   pattern = [{'DEP':'ROOT'}, 
#             {'DEP':'prep','OP':"?"},
#             {'DEP':'agent','OP':"?"},  
#             {'POS':'ADJ','OP':"?"}] 

#   matcher.add("matching_1", None, pattern) 

#   matches = matcher(doc)
#   k = len(matches) - 1

#   span = doc[matches[k][1]:matches[k][2]] 

#   return(span.text)

In [ ]:
import pandas as pd
import re

import spacy
nlp=spacy.load('en_core_web_sm')

def get_entity_pairs(text, coref=True):
    # preprocess text
    text = re.sub(r'\n+', '.', text)  # replace multiple newlines with period
    text = re.sub(r'\[\d+\]', ' ', text)  # remove reference numbers
    text = nlp(text)
    
    def refine_ent(ent, sent):
        unwanted_tokens = (
            'PRON',  # pronouns
            'PART',  # particle
            'DET',  # determiner
            'SCONJ',  # subordinating conjunction
            'PUNCT',  # punctuation
            'SYM',  # symbol
            'X',  # other
        )
        ent_type = ent.ent_type_  # get entity type
        if ent_type == '':
            ent_type = 'NOUN_CHUNK'
            ent = ' '.join(str(t.text) for t in
                           nlp(str(ent)) if t.pos_
                           not in unwanted_tokens and t.is_stop == False)
        elif ent_type in ('NOMINAL', 'CARDINAL', 'ORDINAL') and str(ent).find(' ') == -1:
            refined = ''
            for i in range(len(sent) - ent.i):
                if ent.nbor(i).pos_ not in ('VERB', 'PUNCT'):
                    refined += ' ' + str(ent.nbor(i))
                else:
                    ent = refined.strip()
                    break

        return ent, ent_type

    sentences = [sent.string.strip() for sent in text.sents]  # split text into sentences
    ent_pairs = []
    for sent in sentences:
        sent = nlp(sent)
        spans = list(sent.ents) + list(sent.noun_chunks)  # collect nodes
        spans = spacy.util.filter_spans(spans)
        with sent.retokenize() as retokenizer:
            [retokenizer.merge(span, attrs={'tag': span.root.tag,
                                            'dep': span.root.dep}) for span in spans]
        deps = [token.dep_ for token in sent]

        # limit our example to simple sentences with one subject and object
        if (deps.count('obj') + deps.count('dobj')) != 1\
                or (deps.count('subj') + deps.count('nsubj')) != 1:
            continue

        for token in sent:
            if token.dep_ not in ('obj', 'dobj'):  # identify object nodes
                continue
            subject = [w for w in token.head.lefts if w.dep_
                       in ('subj', 'nsubj')]  # identify subject nodes
            if subject:
                subject = subject[0]
                # identify relationship by root dependency
                relation = [w for w in token.ancestors if w.dep_ == 'ROOT']
                if relation:
                    relation = relation[0]
                    # add adposition or particle to relationship
#                     if relation.nbor(1).pos_ in ('ADP', 'PART'):
#                         relation = ' '.join((str(relation), str(relation.nbor(1))))
                else:
                    relation = 'unknown'

                subject, subject_type = refine_ent(subject, sent)
                token, object_type = refine_ent(token, sent)

                ent_pairs.append([str(subject), str(relation), str(token),
                                  str(subject_type), str(object_type)])

    ent_pairs = [sublist for sublist in ent_pairs
                          if not any(str(ent) == '' for ent in sublist)]
    pairs = pd.DataFrame(ent_pairs, columns=['subject', 'relation', 'object',
                                             'subject_type', 'object_type'])
    print('Entity pairs extracted:', str(len(ent_pairs)))

    return pairs

In [ ]:
nlp.max_length=1000000000

In [ ]:
pairs=get_entity_pairs(text_list)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
print(pairs)

In [ ]:
pairs['relation'].value_counts()[:50]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt


def draw_kg(pairs):
    k_graph = nx.from_pandas_edgelist(pairs, 'subject', 'object',
            create_using=nx.MultiDiGraph())
    node_deg = nx.degree(k_graph)
    layout = nx.spring_layout(k_graph, k=0.15, iterations=20)
    plt.figure(num=None, figsize=(120, 90), dpi=80)
    nx.draw_networkx(
        k_graph,
        node_size=[int(deg[1]) * 500 for deg in node_deg],
        arrowsize=20,
        linewidths=1.5,
        pos=layout,
        edge_color='red',
        edgecolors='black',
        node_color='white',
        )
    labels = dict(zip(list(zip(pairs.subject, pairs.object)),
                  pairs['relation'].tolist()))
    nx.draw_networkx_edge_labels(k_graph, pos=layout, edge_labels=labels,
                                 font_color='red')
    plt.axis('off')
    plt.show()

In [ ]:

# def draw_kg(pairs,c1='red',c2='blue',c3='yellow'):
#     k_graph = nx.from_pandas_edgelist(pairs, 'subject', 'object',
#             create_using=nx.MultiDiGraph())
  
#     node_deg = nx.degree(k_graph)
#     degrees= list(k_graph.out_degree())
#     print(degrees)
#     layout = nx.spring_layout(k_graph, k=0.15, iterations=20)
#     plt.figure(num=None, figsize=(50, 40), dpi=80)
#     nx.draw_networkx(
#         k_graph,
#         node_size=[int(deg[1]) * 500 for deg in node_deg],
#         arrowsize=20,
#         linewidths=1.5,
#         pos=layout,
#         edge_color=c1,
#         edgecolors=c2,
#         node_color=c3,
#         )
#     labels = dict(zip(list(zip(pairs.subject, pairs.object)),
#                   pairs['relation'].tolist()))
#     nx.draw_networkx_edge_labels(k_graph, pos=layout, edge_labels=labels,
#                                  font_color='red')
#     plt.axis('off')
#     plt.show()

**A directed multigraph network is created with nodes sized in proportion to degree centrality.**

In [ ]:
draw_kg(pairs)

Performing Markov Clustering on Knowledge Graph

In [ ]:
!pip install markov_clustering[drawing]

In [ ]:
import markov_clustering as mcl

In [ ]:
k_graph = nx.from_pandas_edgelist(pairs, 'subject', 'object',
            create_using=nx.MultiDiGraph())
adj_matrix = nx.to_numpy_matrix(k_graph)
res = mcl.run_mcl(adj_matrix)
clusters = mcl.get_clusters(res)

In [ ]:
print(clusters)

In [ ]:
def MCL(pairs):
    k_graph = nx.from_pandas_edgelist(pairs, 'subject', 'object',
            create_using=nx.MultiDiGraph())
    adj_matrix = nx.to_numpy_matrix(k_graph)
    res = mcl.run_mcl(adj_matrix)
    clusters = mcl.get_clusters(res)
#     mcl.drawing.draw_graph(adj_matrix, clusters, edge_color="red",node_size=40, with_labels=True)
    mcl.draw_graph(adj_matrix, clusters, node_size=150, with_labels=True, edge_color="silver")

In [ ]:
MCL(pairs)